In [1]:
import copy
from pathlib import Path

import numpy as np
import pandas as pd
import nbconvert

In [ ]:
from tiled.client import from_uri
client = from_uri("http://localhost:8000/api")

In [5]:

import os
from pathlib import Path


DATA_PATH = Path("aimmdb/data/feff/65272_C_007")
print("Data Path:", DATA_PATH)

contents = os.listdir(DATA_PATH)
print("Contents:", contents)


Data Path: aimmdb\data\feff\65272_C_007


FileNotFoundError: [WinError 3] The system cannot find the path specified: 'aimmdb\\data\\feff\\65272_C_007'

In [ ]:
from aimmdb.ingest.load_FEFF_Data import load_feff_data

data, metadata = load_feff_data(DATA_PATH)

In [7]:
def ingest_feff(client, df, verbose=False):
    """
    Upload the FEFF dataset to database
    """

    for (name, prep), g in df.groupby(["sample.name", "sample.prep"]):
        if verbose:
            print(f"{name}: {prep}, {len(g)}")

        sample_id = client.write_sample({"name" : name, "prep" : prep})

        for i, row in g.iterrows():
            feff_df, _ = load_feff_data.dat(row.file)
            metadata = row.metadata
            metadata["dataset"] = "feff"
            metadata["sample_id"] = sample_id
            client["uid"].write_dataframe(feff_df, metadata=metadata, specs=["FEFF"])

In [9]:
print("starting ingestion...")
ingest_feff(client, load_feff_data.data, verbose=True)
print("finished.")

starting ingestion...


NameError: name 'client' is not defined

In [12]:
import aimmdb.schemas as schemas

schemas.FEFFInputMetadata.schema()

{'title': 'FEFFInputMetadata',
 'type': 'object',
 'properties': {'element': {'$ref': '#/definitions/XDIElement'},
  'measurement_type': {'default': 'FEFF',
   'allOf': [{'$ref': '#/definitions/MeasurementEnum'}]},
  'dataset': {'title': 'Dataset', 'type': 'string'},
  'sample_id': {'title': 'Sample Id', 'type': 'string'},
  'input_script': {'title': 'Input Script', 'type': 'string'}},
 'required': ['element', 'dataset', 'sample_id', 'input_script'],
 'definitions': {'XDIElement': {'title': 'XDIElement',
   'type': 'object',
   'properties': {'symbol': {'title': 'Symbol', 'type': 'string'},
    'edge': {'title': 'Edge', 'type': 'string'}},
   'required': ['symbol', 'edge']},
  'MeasurementEnum': {'title': 'MeasurementEnum',
   'description': 'An enumeration.',
   'enum': ['xas', 'rixs'],
   'type': 'string'}}}

In [13]:
import aimmdb.schemas as schemas

schemas.FEFFOutputMetadata.schema()

{'title': 'FEFFOutputMetadata',
 'type': 'object',
 'properties': {'element': {'$ref': '#/definitions/XDIElement'},
  'measurement_type': {'default': 'FEFF',
   'allOf': [{'$ref': '#/definitions/MeasurementEnum'}]},
  'dataset': {'title': 'Dataset', 'type': 'string'},
  'sample_id': {'title': 'Sample Id', 'type': 'string'},
  'output_log': {'title': 'Output Log', 'type': 'string'}},
 'required': ['element', 'dataset', 'sample_id', 'output_log'],
 'definitions': {'XDIElement': {'title': 'XDIElement',
   'type': 'object',
   'properties': {'symbol': {'title': 'Symbol', 'type': 'string'},
    'edge': {'title': 'Edge', 'type': 'string'}},
   'required': ['symbol', 'edge']},
  'MeasurementEnum': {'title': 'MeasurementEnum',
   'description': 'An enumeration.',
   'enum': ['xas', 'rixs'],
   'type': 'string'}}}

In [14]:

import aimmdb.schemas as schemas
schemas.FEFFcards.schema()

{'title': 'FEFFcards',
 'type': 'object',
 'properties': {'atoms': {'title': 'Atoms', 'type': 'number'},
  'control': {'title': 'Control', 'type': 'integer'},
  'exchange': {'title': 'Exchange', 'type': 'number'},
  'title': {'title': 'Title', 'type': 'string'},
  'rpath': {'title': 'Rpath', 'type': 'integer'},
  'potentials': {'title': 'Potentials', 'type': 'number'},
  'xanes': {'title': 'Xanes', 'type': 'number'},
  'edge': {'title': 'Edge', 'type': 'string'},
  'scf': {'title': 'Scf', 'type': 'number'},
  'fms': {'title': 'Fms', 'type': 'number'},
  'S02': {'title': 'S02', 'type': 'number'},
  'corehole': {'title': 'Corehole', 'type': 'string'}},
 'required': ['atoms',
  'control',
  'exchange',
  'rpath',
  'potentials',
  'xanes',
  'edge',
  'scf',
  'fms',
  'S02',
  'corehole']}

In [ ]:
client["uid"]

In [ ]:
# with the correct metadata we can write to the server
# NOTE this doesn't prevent you from writing garbage but does help
df = pd.DataFrame({"a" : np.random.rand(100), "b" : np.random.rand(100)})
metadata = {"dataset" : "feff", "foo" : "bar", "element" : {"symbol" : "Au", "edge" : "K"}, "facility" : {"name" : "ALS"}, "beamline" : {"name" : "8.0.1"}}
node = client["uid"].write_dataframe(df, metadata=metadata, specs=["FEFF"])
node

In [4]:
# clear outputs
!jupyter nbconvert --ClearOutputPreprocessor.enabled=True --inplace ingest_FEFF.ipynb

[NbConvertApp] Converting notebook ingest_FEFF.ipynb to notebook
[NbConvertApp] Writing 4109 bytes to ingest_FEFF.ipynb
